In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import Ridge
import json
import itertools
from tqdm import tqdm

In [2]:
def lorenz_deriv(state, t, sigma=10.0, rho=28.0, beta=8.0/3.0):
    x, y, z = state
    dxdt = sigma * (y - x)
    dydt = x*(rho - z) - y
    dzdt = x*y - beta*z
    return [dxdt, dydt, dzdt]

def generate_lorenz_data(
    initial_state=[1.0, 1.0, 1.0],
    tmax=25.0,
    dt=0.01,
    sigma=10.0,
    rho=28.0,
    beta=8.0/3.0
):
    num_steps = int(tmax / dt) + 1 # +1 to include t=0
    t_vals = np.linspace(0, tmax, num_steps)
    sol = odeint(lorenz_deriv, initial_state, t_vals, args=(sigma, rho, beta))
    return t_vals, sol

In [3]:
def evaluate_nrmse(all_preds, test_target, horizons):
    """
    Evaluate model performance over multiple prediction horizons
    for teacher-forced single-step forecasting or autoregressive rollout.
    """
    horizon_nrmse = {}
    for horizon in horizons:
        preds = all_preds[:horizon]
        targets = test_target[:horizon]
        squared_errors = (preds - targets) ** 2
        variance = np.var(targets, axis=0)
        variance[variance == 0] = 1e-8  # avoid divide-by-zero
        nrmse = np.sqrt(np.sum(squared_errors) / (horizon * np.sum(variance)))
        horizon_nrmse[horizon] = nrmse
    return horizon_nrmse

In [4]:
def compute_valid_prediction_time(y_true, y_pred, t_vals, threshold, lambda_max, dt):
    """
    Compute the Valid Prediction Time (VPT) and compare it to Lyapunov time T_lambda = 1 / lambda_max.
    
    Parameters
    ----------
    y_true : ndarray of shape (N, dim)
        True trajectory over time.
    y_pred : ndarray of shape (N, dim)
        Model's predicted trajectory over time (closed-loop).
    t_vals : ndarray of shape (N,)
        Time values corresponding to the trajectory steps.
    threshold : float, optional
        The error threshold, default is 0.4 as in your snippet.
    lambda_max : float, optional
        Largest Lyapunov exponent. Default=0.9 for Lorenz.
        
    Returns
    -------
    T_VPT : float
        Valid prediction time. The earliest time at which normalized error surpasses threshold
        (or the last time if never surpassed).
    T_lambda : float
        Lyapunov time = 1 / lambda_max
    ratio : float
        How many Lyapunov times the model prediction remains valid, i.e. T_VPT / T_lambda.
    """
    # 1) Average of y_true
    y_mean = np.mean(y_true, axis=0)  # shape (dim,)
    
    # 2) Time-averaged norm^2 of (y_true - y_mean)
    y_centered = y_true - y_mean
    denom = np.mean(np.sum(y_centered**2, axis=1))  # scalar
    
    # 3) Compute the normalized error delta_gamma(t) = ||y_true - y_pred||^2 / denom
    diff = y_true - y_pred
    err_sq = np.sum(diff**2, axis=1)  # shape (N,)
    delta_gamma = err_sq / denom      # shape (N,)
    
    # 4) Find the first time index where delta_gamma(t) exceeds threshold
    idx_exceed = np.where(delta_gamma > threshold)[0]
    if len(idx_exceed) == 0:
        # never exceeds threshold => set T_VPT to the final time
        T_VPT = t_vals[-1]
    else:
        T_VPT = t_vals[idx_exceed[0]]
    
    # 5) Compute T_lambda and ratio
    T_lambda = 1.0 / lambda_max

    # print(f"\n--- Valid Prediction Time (VPT) with threshold={threshold}, lambda_max={lambda_max} ---")

    T_VPT = (T_VPT - t_vals[0])  # Adjust T_VPT to be relative to the start time
    ratio = T_VPT / T_lambda

    return T_VPT, T_lambda, ratio

In [5]:
def compute_attractor_deviation(predictions, targets, cube_size=(0.1, 0.1, 0.1)):
    """
    Compute the Attractor Deviation (ADev) metric.

    Parameters:
        predictions (numpy.ndarray): Predicted trajectories of shape (n, 3).
        targets (numpy.ndarray): True trajectories of shape (n, 3).
        cube_size (tuple): Dimensions of the cube (dx, dy, dz).

    Returns:
        float: The ADev metric.
    """
    # Define the cube grid based on the range of the data and cube size
    min_coords = np.min(np.vstack((predictions, targets)), axis=0)
    max_coords = np.max(np.vstack((predictions, targets)), axis=0)

    # Create a grid of cubes
    grid_shape = ((max_coords - min_coords) / cube_size).astype(int) + 1

    # Initialize the cube occupancy arrays
    pred_cubes = np.zeros(grid_shape, dtype=int)
    target_cubes = np.zeros(grid_shape, dtype=int)

    # Map trajectories to cubes
    pred_indices = ((predictions - min_coords) / cube_size).astype(int)
    target_indices = ((targets - min_coords) / cube_size).astype(int)

    # Mark cubes visited by predictions and targets
    for idx in pred_indices:
        pred_cubes[tuple(idx)] = 1
    for idx in target_indices:
        target_cubes[tuple(idx)] = 1

    # Compute the ADev metric
    adev = np.sum(np.abs(pred_cubes - target_cubes))
    return adev

In [6]:
import numpy as np
from sklearn.linear_model import Ridge


class GliaNeuronTripartiteReservoirESN:
    """
    Glia–Neuron Tripartite-Synapse Reservoir (GNT-SR)

    ------------------------------------------------------------------
    State variables
        x  ∈ ℝᴺ   : neuronal membrane potentials (fast)
        c  ∈ ℝᴹ   : astrocytic Ca²⁺ concentrations   (intermediate)
        g  ∈ ℝᴹ   : gliotransmitter release fractions (slow)

    Update order per time-step t → t+1
        1)  c     ←   (1-μ)·c   + μ·η·σ(W_ng x − θ)   + μ·D·L_g c
        2)  g     ←   γ·c / (1 + c)
        3)  x     ←  (1-α)·x + α·tanh(W_nn x + W_in u + W_gn g)

    Echo-state property is guaranteed by fixing ρ(W_nn) < 1 and because
    astrocytic modulation acts only as an added bias.
    """

    # ------------------------------------------------------------------
    #                          constructor                              
    # ------------------------------------------------------------------
    def __init__(
        self,
        reservoir_size: int = 800,      # N  neurons
        n_astrocytes: int | None = None,  # M  astrocytes; default N//10
        input_dim: int = 3,
        rho_star: float = 0.8,          # desired spectral radius of W_nn
        alpha: float = 0.5,             # neuronal leak        (α)
        tau_c: float = 200.0,           # Ca²⁺ relaxation time (steps)
        eta: float = 1.0,               # Ca²⁺ activation gain (η)
        D_diff: float = 0.005,          # astrocytic diffusion (D)
        theta: float = 0.0,             # Ca²⁺ threshold       (θ)
        gamma_glia: float = 0.4,        # max gliotransmitter  (γ)
        input_scale: float = 0.5,
        ridge_alpha: float = 1e-6,
        use_quadratic_readout: bool = True,
        seed: int = 42,
    ):
        # -------------- dimensions ------------------------------------
        self.N = reservoir_size
        self.M = n_astrocytes if n_astrocytes is not None else max(1, reservoir_size // 10)
        self.d_in = input_dim

        # -------------- hyper-parameters ------------------------------
        self.rho_star = rho_star
        self.alpha = alpha
        self.mu = 1.0 / tau_c           # μ = Δt / τ_c  (Δt = 1)
        self.eta = eta
        self.D = D_diff
        self.theta = theta
        self.gamma = gamma_glia
        self.input_scale = input_scale
        self.ridge_alpha = ridge_alpha
        self.use_quad = use_quadratic_readout
        self.seed = seed

        rng = np.random.default_rng(self.seed)

        # --------------------------------------------------------------
        # static weight matrices
        # --------------------------------------------------------------
        # 1) Neuron-neuron recurrent matrix   W_nn
        W_raw = rng.standard_normal((self.N, self.N)).astype(np.float32)
        eig_max = np.max(np.abs(np.linalg.eigvals(W_raw)))
        self.W_nn = (self.rho_star / eig_max) * W_raw

        # 2) Input weights  W_in
        self.W_in = (
            rng.uniform(-1.0, 1.0, size=(self.N, self.d_in)) * self.input_scale
        ).astype(np.float32)

        # 3) Neuron → astrocyte  W_ng   (rows sparse, non-neg)
        self.W_ng = rng.uniform(0.0, 1.0, size=(self.M, self.N)).astype(np.float32)
        # 4) Astrocyte → neuron  W_gn   (columns sparse, non-neg)
        self.W_gn = rng.uniform(0.0, 1.0, size=(self.N, self.M)).astype(np.float32)

        # 5) Laplacian L_g  for astrocyte diffusion (1-D chain for simplicity)
        L = np.zeros((self.M, self.M), dtype=np.float32)
        for i in range(self.M):
            if i > 0:
                L[i, i - 1] = -1.0
            if i < self.M - 1:
                L[i, i + 1] = -1.0
            L[i, i] = (2.0 if 0 < i < self.M - 1 else 1.0)
        self.L_g = L

        # --------------------------------------------------------------
        # dynamic state vectors
        # --------------------------------------------------------------
        self.x = np.zeros(self.N, dtype=np.float32)
        self.c = np.zeros(self.M, dtype=np.float32)
        self.g = np.zeros(self.M, dtype=np.float32)

        self.W_out: np.ndarray | None = None

    # ------------------------------------------------------------------
    #                            helpers                                
    # ------------------------------------------------------------------
    def reset_state(self):
        self.x.fill(0.0)
        self.c.fill(0.0)
        self.g.fill(0.0)

    def _step(self, u_t: np.ndarray):
        """One full tripartite update."""
        # ---- astrocytic Ca²⁺ -----------------------------------------
        pre_c = self.W_ng @ self.x - self.theta
        sigma_term = 1.0 / (1.0 + np.exp(-pre_c))        # logistic σ
        diff_term = self.D * (self.L_g @ self.c)
        self.c = (1.0 - self.mu) * self.c + self.mu * (self.eta * sigma_term + diff_term)

        # ---- gliotransmitter release --------------------------------
        self.g = self.gamma * self.c / (1.0 + self.c)    # element-wise

        # ---- neuronal update ----------------------------------------
        bias = self.W_gn @ self.g                        # N-vector
        pre = self.W_nn @ self.x + self.W_in @ u_t + bias
        x_new = np.tanh(pre).astype(np.float32)
        self.x = (1.0 - self.alpha) * self.x + self.alpha * x_new

    # ------------------------------------------------------------------
    #                       read-out training                           
    # ------------------------------------------------------------------
    def fit_readout(self, inputs: np.ndarray, targets: np.ndarray, discard: int = 100):
        """
        Teacher forcing to train W_out.

        inputs  : [T, d_in]
        targets : [T, d_out]
        """
        T, d_in = inputs.shape
        if d_in != self.d_in:
            raise ValueError("input_dim mismatch")
        if T <= discard + 1:
            raise ValueError("sequence too short")

        self.reset_state()
        feats_list = []
        for t in range(T):
            self._step(inputs[t])
            if t >= discard:
                if self.use_quad:
                    feats = np.concatenate(
                        [
                            self.x,
                            self.x * self.x,
                            self.c,
                            self.g,
                            [1.0],
                        ]
                    )
                else:
                    feats = np.concatenate([self.x, self.c, self.g, [1.0]])
                feats_list.append(feats)

        X_feat = np.asarray(feats_list, dtype=np.float32)
        Y = targets[discard:]

        reg = Ridge(alpha=self.ridge_alpha, fit_intercept=False)
        reg.fit(X_feat, Y)
        self.W_out = reg.coef_.astype(np.float32)

    # ------------------------------------------------------------------
    #                     autoregressive rollout                        
    # ------------------------------------------------------------------
    def predict_autoregressive(self, init_input: np.ndarray, n_steps: int) -> np.ndarray:
        """
        Free-run the reservoir for n_steps.

        init_input : shape (d_in,)
        returns    : shape (n_steps, d_out)
        """
        if self.W_out is None:
            raise RuntimeError("Call fit_readout() first")

        d_out = self.W_out.shape[0]
        preds = np.empty((n_steps, d_out), dtype=np.float32)

        #self.reset_state()
        u_t = init_input.astype(np.float32).copy()

        for t in range(n_steps):
            self._step(u_t)

            if self.use_quad:
                feat_vec = np.concatenate([self.x, self.x * self.x, self.c, self.g, [1.0]])
            else:
                feat_vec = np.concatenate([self.x, self.c, self.g, [1.0]])

            y_t = (self.W_out @ feat_vec).astype(np.float32)
            preds[t] = y_t
            u_t = y_t[: self.d_in]      # feedback first d_in components

        return preds

In [7]:
grid = {
    "reservoir_size": [300],
    "n_astrocytes": [20],
    "input_dim": [3],
    "rho_star": [0.98],
    "alpha": [0.4],
    "tau_c": [200.0],
    "eta": [1.2],
    "D_diff": [0.07],
    "theta" : [0.02],
    "gamma_glia": [0.01],
    "input_scale" : [1.3],
    "ridge_alpha" : [1e-8],
    "use_quadratic_readout" : [True],
}

In [8]:
def run_grid_search(model_class, param_grid, model_name,
                    output_path="grid_search_results.json", f=generate_lorenz_data, lambda_max=0.9):
    combos = list(itertools.product(*param_grid.values()))
    param_keys = list(param_grid.keys())
    print(f"\n== Initial grid search for {model_name} with {len(combos)} combinations ==")

    results = []
    # horizons = list(range(10, 1001, 10))
    # horizons = [200, 400, 600, 800, 1000]
    

    for comb in tqdm(combos, desc="Grid Search"):
        params = dict(zip(param_keys, comb))
        seed_scores_vpt = []
        # horizon_nrmse_all = {h: [] for h in horizons}
        # horizon_nrmse_all_open_loop = {h: [] for h in horizons}
        # adev_scores = []
        # ldev_scores = []

        for initial_state in [[1.0, 1.0, 1.0], [1.0, 2.0, 3.0], [2.0, 1.5, 4.0]]:
            tmax = 250
            dt = 0.02
            t_vals, lorenz_traj = f(
                initial_state=initial_state,
                tmax=tmax,
                dt=dt
            )

            washout = 2000
            t_vals = t_vals[washout:]
            lorenz_traj = lorenz_traj[washout:]

            scaler = MinMaxScaler()
            scaler.fit(lorenz_traj)
            lorenz_traj = scaler.transform(lorenz_traj)

            T_data = len(lorenz_traj)
            for train_frac in [0.7, 0.75, 0.8]:
                train_end = int(train_frac * (T_data - 1))
                train_input = lorenz_traj[:train_end]
                train_target = lorenz_traj[1:train_end + 1]
                test_input = lorenz_traj[train_end:-1]
                test_target = lorenz_traj[train_end + 1:]
                n_test_steps = len(test_input)
                initial_in = test_input[0]

                for seed in np.arange(1, 6):
                    model = model_class(**params, seed=seed)
                    model.fit_readout(train_input, train_target, discard=100)
                    preds = model.predict_autoregressive(initial_in, n_test_steps)
                    
                    T_VPT_s, _, ratio = compute_valid_prediction_time(test_target, preds, t_vals, 0.4, lambda_max, dt)
                    seed_scores_vpt.append(ratio)

                    # horizon_nrmse = evaluate_nrmse(preds, test_target, horizons)
                    # for h in horizons:
                    #     horizon_nrmse_all[h].append(horizon_nrmse[h])


                    # adev = compute_attractor_deviation(preds, test_target)
                    # adev_scores.append(adev)

                    # ldev = compute_lyapunov_exponent("Lorenz", preds, dt)
                    # ldev_scores.append(ldev)

        mean_vpt = float(np.mean(seed_scores_vpt))
        std_vpt = float(np.std(seed_scores_vpt))
        # mean_nrmse_dict = {str(h): float(np.mean(horizon_nrmse_all[h])) for h in horizons}
        # std_nrmse_dict  = {str(h): float(np.std(horizon_nrmse_all[h]))  for h in horizons}
        # mean_adev = float(np.mean(adev_scores))
        # std_adev = float(np.std(adev_scores))
        # mean_ldev = float(np.mean(ldev_scores))
        # std_ldev = float(np.std(ldev_scores))

        results.append({
            "params": params,
            "seed_scores_T_VPT": seed_scores_vpt,
            "mean_T_VPT": mean_vpt,
            "std_T_VPT": std_vpt,
            # "mean_NRMSEs": mean_nrmse_dict,
            # "std_NRMSEs": std_nrmse_dict,
            # "mean_ADev": mean_adev,
            # "std_ADev": std_adev,
        })

    with open(output_path, "w") as f:
        json.dump(results, f, indent=2)
    print(f"\nAll results saved to `{output_path}`")

    return results

In [9]:
run_grid_search(GliaNeuronTripartiteReservoirESN, grid, "GliaNeuronTripartiteReservoir", output_path="GLIA.json", f=generate_lorenz_data, lambda_max=0.9)


== Initial grid search for GliaNeuronTripartiteReservoir with 1 combinations ==


Grid Search: 100%|██████████| 1/1 [00:48<00:00, 48.73s/it]


All results saved to `GLIA.json`


[{'params': {'reservoir_size': 300,
   'n_astrocytes': 20,
   'input_dim': 3,
   'rho_star': 0.98,
   'alpha': 0.4,
   'tau_c': 200.0,
   'eta': 1.2,
   'D_diff': 0.07,
   'theta': 0.02,
   'gamma_glia': 0.01,
   'input_scale': 1.3,
   'ridge_alpha': 1e-08,
   'use_quadratic_readout': True},
  'seed_scores_T_VPT': [np.float64(9.702),
   np.float64(9.738),
   np.float64(9.612),
   np.float64(7.776),
   np.float64(9.720000000000004),
   np.float64(7.938),
   np.float64(9.828000000000001),
   np.float64(8.009999999999998),
   np.float64(11.304000000000002),
   np.float64(7.847999999999999),
   np.float64(10.674),
   np.float64(10.692000000000002),
   np.float64(8.730000000000002),
   np.float64(8.730000000000002),
   np.float64(8.856000000000003),
   np.float64(10.638),
   np.float64(8.1),
   np.float64(8.064),
   np.float64(10.620000000000003),
   np.float64(8.028),
   np.float64(11.195999999999998),
   np.float64(9.306000000000003),
   np.float64(11.214),
   np.float64(9.233999999999998